In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import SelectKBest, SelectPercentile, mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


In [2]:
df = pd.read_csv('selected_features.csv')
df = df.sample(frac = 1, random_state=42)

X = df.loc[:,['verb_count',
 'proper_noun_count',
 'emotiveness',
 'compound',
 'neutral']]

feature_names = list(X.columns)

y=np.array(df.label)
X = np.array(X)


In [3]:
#Split X & y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=0.2, shuffle = True, random_state = 8)
#Split remaining train into train and  validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
    test_size=0.25, random_state= 8) # 0.25 x 0.8 = 0.2


print("X_train shape: {} \n".format(X_train.shape))
print("X_test shape: {} \n" .format(X_test.shape))
print("X_val shape: {} \n".format(X_val.shape))
print("y_train shape: {}\n".format(y_train.shape))
print("y_test shape: {}\n".format(y_test.shape))
print("y_val shape: {}\n".format(y_val.shape))


# Scale the featrues, you are fitting and transforming the train features but not the test/val. Testing data must remain unseen by the model. If you fit the testing data then your model learned on the testing data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

X_train shape: (26923, 5) 

X_test shape: (8975, 5) 

X_val shape: (8975, 5) 

y_train shape: (26923,)

y_test shape: (8975,)

y_val shape: (8975,)



In [4]:
rfc = RandomForestClassifier()

In [5]:
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [6]:
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [7]:
y_pred = rfc.predict(X_val)
print(classification_report(y_pred, y_val))

              precision    recall  f1-score   support

           0       0.85      0.81      0.83      4435
           1       0.82      0.86      0.84      4540

    accuracy                           0.83      8975
   macro avg       0.84      0.83      0.83      8975
weighted avg       0.84      0.83      0.83      8975



In [11]:
max_features = ['sqrt', 'log2', None]
min_samples_leaf = [1,2,3,4,5]
n_jobs = -1
criterion = ['gini', 'entropy']
n_estimators = [10,20,30,40,50,60,70,80,100]

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
param_grid = dict(max_features=max_features, min_samples_leaf=min_samples_leaf,
                 criterion=criterion, n_estimators=n_estimators)

In [12]:
gridCVForest = GridSearchCV(rfc, param_grid, n_jobs=-1, scoring='accuracy', cv=cv
                           ,error_score='raise', verbose=1)

grid_fit=gridCVForest.fit(X_train, y_train)

#Best parameters and accuracy
print("Best: %f using %s" % (grid_fit.best_score_, grid_fit.best_params_))

Fitting 9 folds for each of 270 candidates, totalling 2430 fits
Best: 0.826481 using {'criterion': 'gini', 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 100}


In [13]:
y_pred = gridCVForest.predict(X_val)
print(classification_report(y_pred, y_val))

              precision    recall  f1-score   support

           0       0.86      0.80      0.83      4488
           1       0.81      0.86      0.84      4487

    accuracy                           0.83      8975
   macro avg       0.83      0.83      0.83      8975
weighted avg       0.83      0.83      0.83      8975



In [14]:
y_pred= gridCVForest.predict(X_test)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.87      0.80      0.83      4645
           1       0.80      0.87      0.83      4330

    accuracy                           0.83      8975
   macro avg       0.83      0.83      0.83      8975
weighted avg       0.83      0.83      0.83      8975



In [16]:
pl = pd.read_csv('political news for cv')
gp = pd.read_csv('gossip news for cv')

In [17]:
gp = gp.sample(frac = 1, random_state=42)
pl = pl.sample(frac = 1, random_state=42)

In [18]:

gp = gp.sample(frac = 1, random_state=42)

X = gp.loc[:,['verb_count',
 'proper_noun_count',
 'emotiveness',
 'compound',
 'neutral']]

feature_names = list(X.columns)

y=np.array(gp.label)
X = np.array(X)

sc = StandardScaler()
X = sc.fit_transform(X)


In [19]:
y_pred= gridCVForest.predict(X)
print(classification_report(y_pred, y))

              precision    recall  f1-score   support

           0       0.30      0.81      0.44      6143
           1       0.78      0.26      0.39     15479

    accuracy                           0.42     21622
   macro avg       0.54      0.54      0.42     21622
weighted avg       0.64      0.42      0.40     21622



In [20]:
X = pl.loc[:,['verb_count',
 'proper_noun_count',
 'emotiveness',
 'compound',
 'neutral']]

feature_names = list(X.columns)

y=np.array(pl.label)
X = np.array(X)

sc = StandardScaler()
X = sc.fit_transform(X)


In [21]:
y_pred= gridCVForest.predict(X)
print(classification_report(y_pred, y))

              precision    recall  f1-score   support

           0       0.29      0.78      0.43       169
           1       0.90      0.51      0.65       641

    accuracy                           0.57       810
   macro avg       0.60      0.64      0.54       810
weighted avg       0.77      0.57      0.60       810

